# Introduction

This tutorial will introduce Aho–Corasick(AC) algorithm, this algorithm is used to searching specific strings in a text, given pattern set. 

Let’s say pattern set is {his, he, hers, she}, and text is “hershershershers”, then we use Aho–Corasick algorithm to get positions of patterns occurring at text, i.g.:

1 ['he']
<br>
3 ['hers']
<br>
5 ['she', 'he']
<br>
7 ['hers']
<br>
9 ['she', 'he']
<br>
11 ['hers']
<br>
13 ['she', 'he']
<br>
15 ['hers']

Number indicates the position of the last letter, followed list indicates which patterns match at this position, e.g. 5 ['she', 'he'] means that 'she' and 'he' occur and their ‘e’ are the 5th letter at original string ‘hershershershers’ (indexing from 0).

In data science, manipulating string and searching for patterns are fundamental, we often process tons of text data, searching strings is a very basic but important operation. For example, occurrence frequency of a specific pattern in corpus can be a feature in machine learning algorithm, and we count occurrence by pattern matching. On the other side, we can filter out patterns by searching patterns algorithm.

A famous fast string matching algorithm is Knuth-Morris-Pratt(KMP) algorithm, it matches one pattern with linear complexity. But it only searches for one pattern, when we want to match multiple patterns, its complexity will time number of patterns, which is unacceptable. So, AC algorithm is introduced, providing linear complexity multiple patterns matching.


# Aho–Corasick Algorithm
AC algorithm is invented by Alfred V. Aho and Margaret J. Corasick in 1975[1], and it is the most classic multiple pattern matching algorithm. It’s based on Deterministic Finite Automaton(DFA).

Actually, when pre-processing, we generate three functions: goto, failure and output. 
<br>
Goto function decide state transition when letter matches.
<br>
Failure function decide state transition when letter mismatches.
<br>
Output function decide which patterns to output at current state.

At first, let me demostrate this algorithm using existing goto, failure and output functions. Then I will introduce the way to computer these three functions.

## Example
Following is an example from Alfred V. Aho and Margaret J. Corasick’s original paper[2].

<img src="files/imgs/example.png" width="400">

In this example, pattern set is {his, he, hers, she}. Assuming string we want to search in is “hshe”.

At state 0, ‘h’ matches, so we use goto function to transit to state 1 and consume one letter in “hshe”, now we’re at letter ‘s’. 

State 1 has two ways out ‘e’ and ‘i’, but neither of these match ‘s’, so failure function leads us to state 0 because f(1) = 0.

At state 0, goto function has entrance ‘s’, so we go to state 3 and consume ‘s’ in “hshe”. Then we go to state 4, consuming ‘h’. Then we goto state 5, consuming ‘e’. 

At state 5, we output {she, he} because output function is valid at state 5. At last, we finish processing “hshe”, and output “3, {she, he}”. (3 is index of where we are at the string).

Algorithm ends because we consume all letters.

## Get Three Functions
### Goto Function
Firstly, I will introduce how to generate goto function. Goto(r, a) = s means that state r can go to state s given input a.

Obviously, goto function is be represented as a prefix trie of pattern set. So, we can use dict in python to construct this structure. I add patterns one by one. In order, for every letter in pattern, if this letter exist in dict, go to nested dict, else create a new nested dict. 

At the same time, we can generate output function. I use '|' to indicate the end of pattern and store corresponding pattern in it for further use. When we are searching for patterns, occurrence of '|' means that we can output positions at this state.

In [3]:
root = {}
def add(self, pattern):
    for letter in pattern:
        self = self.setdefault(letter,{})
    self['|'] = pattern

add(root,'his')
add(root,'he')
add(root,'hers')
add(root,'she')

print("Goto function")
for i in root:
    print(i,':', root[i])

Goto function
h : {'i': {'s': {'|': 'his'}}, 'e': {'|': 'he', 'r': {'s': {'|': 'hers'}}}}
s : {'h': {'e': {'|': 'she'}}}


### Failure Function
Fail[state] = next state when letter mismatches.

At first, in trie, we define depth of a node as length of path from root.
 
<img src="files/imgs/trie_.png" width="400"> 
 
In above trie, node 0 has depth of 0, node 1 and 3 have depth of 1, etc. Layer x mean nodes with depth of x.

We will get failure function layer by layer. We define that fail(0) = 0, and all nodes with depth 1 have failure value 0, i.g. fail(1) = fail(3) = 0. Because in layer 0 and layer 1, if we mismatch, we start from the very begining. Then we use recursion to get all layers' failure function.

In order in to get failure function output of state s at depth d, assuming that we have failure functions for node r at d-1 layer:
<br>
<br>
<center>if there is an input a making goto(r, a) = s, then we have fail(s) = goto( f(r), a).</center>

E.g. we have goto(3, h) = 4, so we have fail(4) = goto( f(3) , h) = goto(0, h) = 1

Because when we fail at layer d, we succeed at every letter from root to current letter, in this example, we match ‘sh’, though we fail to extend this string, we still have chance to match suffix of current matched string, in the other words, there may be other paths matching this suffix ‘h’, failure function indicates possible match of this suffix.

### Output Function
Output function indicates which patterns are matched at current node, e.g. at node 5, there’re two matches: ‘she’ and ‘he’. Generate output function is very straightforward, when we construct prefix trie, when add patterns one by one, at the same time, we append current pattern to output set of the end node of current pattern. E.g. we append ‘she’ at node 5, and we append 'he' at node 5.

<br>
<br>
But there’s a problem here in practice, in our trie, most entrances are invalid, so when amount of patterns becomes very large, nested dictionaries can be very memory-consuming. Instead, we can optimize memory usage by using double array trie[3] to represent this structure.

## Double Array Trie
There’re three tables in double array trie: Next table, Base table and Check table.
<br>
If we are in state s, transit to state t given input c, an important formula is:
<center>&t in Next table = &s in Next table + Base[s] + ord(c)   for s[c] -> t</center>
<br>
It means that position of state t in Next table = position of state s in Next table + Base value of s + ascii value of input c. And Check[t] = s is used to indicates valid state transition, if Check[t] != s, even above formula is true, we can not say that state s transit to state t.

Let’s demostrate a double array trie based on this formula. The example is the same as we used above {his, he, hers, she}. 
<br>
Here is the prefix trie. ASCII value h=104; s=115; e=101; i=105; r=114.
<img src="files/imgs/trie.png" width="400">

We update tables layer by layer, the first layer is {h, s}, and the second is {e, i, h}, etc.
<br>
Root has state id 0. For the first layer, there are two new states, we increasingly assign them states id of 1 and 2. Because these two states are transited from state 0, according to the formula, &1(=1) = 0 + Base[0] + ord(h), we get that Base[0] = -103; &2(=12) = 0 + Base[0] + ord(s), Base[0] = -103.

Check[1] = 0; Check[2] = 0.
<center>Next table</center>
<img src="files/imgs/nexttable_first.png" width="700">
We notice that states from the same former states are in the Next table with a distance of difference of their ascii values, &2 - &1 = ord(s) – ord(h).

As for the second layer {e, i, h}, in the same way, assigned them state id 3, 4 and 5. State 3 and state 4 come from state 1(h), so Base[1] = &3(2) – 1 – ord(e) = -100. State 5 comes from state 2, so Base[2] = &5(3) – 12 – ord(h) = -113. 

And we always want to put them in the same Next table to save space, so state 3(e) is right next to state 1(h). If there’s conflict, try next cell, and we allocate 256 more space if needed. You may notice that we can always find the entrance of new state by the formula, whatever the offset is, because Base table varies as we move states of new layer integrally.

Check[3] = 1; Check[4] = 1; Check[5] = 2.
<center>Next table</center>
<img src="files/imgs/nexttable_second.png" width="700">

Similarly, we assign r, s and e state id 6, 7 and 8. Base[3] = -113, Base[4] = -116, Base[5] = -97
Check[6]=3; Check[7]=4; Check[8]=5.
<center>Next table</center>
 <img src="files/imgs/nexttable_third.png" width="700">
At last, we assign s state id 9, Base[6] = -111, Check[9] = 6
 
<center>Final Next table</center>
 <img src="files/imgs/nexttable_final.png" width="700">
 
<center>Base table</center>
 <img src="files/imgs/base.png" width="400">
 
<center>Check table</center>
 <img src="files/imgs/check.png" width="400">
 
<center>Fail function</center>
 <img src="files/imgs/failure.png" width="400">

<center>
output function:
<br>
3 ['he']
<br>
7 ['his']
<br>
8 ['she', 'he']
<br>
9 ['hers']
<br>
</center>
In this way, Base table and Check table only use memory of number of states, and we can put Next table into several times 256 memory cell (ascii has 256 values). You can see in this way, we save a lot of space by put them compactly. Code is followed.



In [4]:
#init next,check,fail tables
#fail table stores states from failure states, fail[i][0] is position in next table, fail[i][1] is state id
check = {}
queue = []
fail = {}
fail[0] = (0,0)
nexttable = {}
nexttable[0] = (0,None)
j = 1
state = 1

#init tables for the first layer of trie
sortedKeys = sorted(root.keys())
for x in sortedKeys:
    position = j + ord(x) - ord(sortedKeys[0])
    nexttable[position] = (state,x) #update next table
    check[state] = 0 #update check table
    fail[state] = (0,0) #update fail function
    queue.append((root[x],position,state))#keep pointer of trie, position in next table, state id
    state += 1
j+=1

#generate base table for the first layer
#position of state t in next table = position of state s in next table + Base value of s + ascii value of input c
base = {}
base[0] = 1 - 0 - ord(nexttable[1][1])  

output = {}

In [5]:
#BFS, update next,base,fail
while len(queue) != 0:
    p = queue[0][0]
    if len(p.keys()) != 1 or list(p.keys())[0] != '|': #if not ends
        base[queue[0][2]] = j - queue[0][1] - ord(sorted(p.keys())[0])
        for x in sorted(p.keys()):
            if x != '|': #for every non-end nodes
                position = j + ord(x) - ord(sorted(p.keys())[0])
                nexttable[position] = (state,x) #update next table
                check[state] = queue[0][2] #update check table
                prfail = fail[queue[0][2]] #state which former node fails and transit to 
                prfail_x = prfail[0] + base[prfail[1]] + ord(x)
                if prfail_x in nexttable and check[nexttable[prfail_x][0]] == prfail[1]:
                    fail[state] = (prfail_x,nexttable[prfail_x][0]) #update fail function of current node
                else:
                    fail[state] = fail[prfail[1]] 
                queue.append((p[x],position,state)) #put current node's childs into queue
                state += 1
            else:
                output[queue[0][2]] = [p['|']]
        while j in nexttable:
            j+=1
    else:
        output[queue[0][2]] = [p['|']]
        if fail[queue[0][2]][1] in output:
            output[queue[0][2]].extend(output[fail[queue[0][2]][1]])
        prfail = fail[queue[0][2]] #state which former node fails and transit to 
        prfail_x = prfail[0] + base[prfail[1]] + ord(x)
        if prfail_x in nexttable and check[nexttable[prfail_x][0]] == prfail[1]:
             fail[state] = (prfail_x,nexttable[prfail_x][0]) #update fail function of current node
        else:
             fail[state] = fail[prfail[1]] 
    del queue[0]
  

After construction, let's print three tables, failure function and output function.

In [6]:
print("nexttable:")
for i in range(max(list(nexttable))+1):
    if i in nexttable:
        print(str(i)+'\t'+str(nexttable[i][0])+'\t'+str(nexttable[i][1]))
    else:
        print(i)

print("\nbase table:")
for i in base:
    print(str(i)+'\t'+ str(base[i]))
    
print("\ncheck table:")
for i in check:
    print(str(i)+'\t'+ str(check[i]))
    
print("\nfail function:")
for i in fail:
    print(str(i)+'\t'+str(fail[i][0])+'\t'+str(fail[i][1]))
    
print("\noutput function:")
for i in output:
    print(i, output[i])

nexttable:
0	0	None
1	1	h
2	3	e
3	5	h
4	6	r
5	7	s
6	4	i
7	8	e
8	9	s
9
10
11
12	2	s

base table:
0	-103
1	-100
2	-113
3	-112
4	-116
5	-97
6	-111

check table:
1	0
2	0
3	1
4	1
5	2
6	3
7	4
8	5
9	6

fail function:
0	0	0
1	0	0
2	0	0
3	0	0
4	0	0
5	1	1
6	0	0
7	12	2
8	2	3
9	12	2
10	0	0

output function:
3 ['he']
7 ['his']
8 ['she', 'he']
9 ['hers']


### Search Algorithm
After construct three tables and functions, now we can use these for matching patterns in strings. Fortunately, searching algorithm is easy as I illustrated in the example. It’s just running a DFA, starting from the first letter of string, use the formula to get the next state, check whether it’s valid by checking check table. At any states, if output function has an entrance, output position and patterns. 

Using “hshe” again.


From state 0, next state is 0 + base[0] + ord(h) = 1(state 1), and check[1] = 0(valid transition), so now we are at state 1(h). 
<br>
Next state is 1 + base[1] + ord(s) = 16 (not entrance), so use fail function to transit to fail[1] = 0, state 0.
<br>
Next state is 0 + base[0] + ord(s) = 12(state 2), and check[2] = 0 (valid), so now we are at state 2(s).
<br>
Next state is 12 + base[2] + ord(h) = 3(state 5), and check[5] = 2(valid), so now we are at state 5(h).
<br>
Next state is 3 + base[5] + ord(e) = 7(state 8), and check[8] = 5(valid), so now we are at state 8(e).
<br>
And output function is valid now, so we output output[8] = ['she', 'he'], position in “hshe” is 3 now, we also output this.


In [9]:
#position of state t in next table = position of state s in next table + Base value of s + ascii value of input c
#search function
def find(word):
    now_pos = 0
    i = 0
    for x in word:
        if nexttable[now_pos][0] in base: 
            next_pos = now_pos + base[nexttable[now_pos][0]] + ord(x)#use formula to transit to next state
            #use failure function to transit state while invalid
            while next_pos not in nexttable or check[nexttable[next_pos][0]] != nexttable[now_pos][0]:
                now_pos = fail[nexttable[now_pos][0]][0]
                next_pos = now_pos + base[nexttable[now_pos][0]] + ord(x)
        else: #leaf node has no base, so directly transit
            now_pos = fail[nexttable[now_pos][0]][0]
            next_pos = now_pos + base[nexttable[now_pos][0]] + ord(x)#use formula to transit to next state
            #use failure function to transit state while invalid
            while next_pos not in nexttable or check[nexttable[next_pos][0]] != nexttable[now_pos][0]: 
                now_pos = fail[nexttable[now_pos][0]][0]
                next_pos = now_pos + base[nexttable[now_pos][0]] + ord(x)
        now_pos = next_pos
        if nexttable[now_pos][0] in output:#if there's an entrance is output function, output it
            print(i,output[nexttable[now_pos][0]])
        i+=1

Let's try to match {his, he, hers, she} in 'hshe' and longer 'hershershershers'!

In [11]:
print('hshe')
find('hshe')
print('-----------')
print('hershershershers')
find('hershershershers')

hshe
3 ['she', 'he']
-----------
hershershershers
1 ['he']
3 ['hers']
5 ['she', 'he']
7 ['hers']
9 ['she', 'he']
11 ['hers']
13 ['she', 'he']
15 ['hers']


# Summary
AC algorithm is used to match multiple patterns in text. Once we finished pre-processing, we can use O(n) time to find all matches in text, n is the length of text. In this tutorial, I use double array trie to represent goto function and failure function, which saves a lot of memory.

# Reference
1.	Aho–Corasick algorithm. (2018, February 28). Retrieved March 31, 2018, from https://en.wikipedia.org/wiki/Aho–Corasick_algorithm
2.	 Aho, Alfred V.; Corasick, Margaret J. (June 1975). "Efficient string matching: An aid to bibliographic search". Communications of the ACM. 18 (6): 333–340. doi:10.1145/360825.360855. MR 0371172.
3. Aoe, J. I., Morimoto, K., & Sato, T. (1992). An efficient implementation of trie structures. Software: Practice and Experience, 22(9), 695-721.
